<a href="https://colab.research.google.com/github/andreagrusso/GPT2_word_saliency_fMRI/blob/main/gpt2%2BGianna_15sContext_NoMultiTokenWords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 2.3 MB/s 
     |████████████████████████████████| 52 kB 1.1 MB/s 
     |████████████████████████████████| 636 kB 36.8 MB/s 
     |████████████████████████████████| 895 kB 27.8 MB/s 
     |████████████████████████████████| 3.3 MB 29.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
from torch.nn import functional as F
from transformers import AutoTokenizer, AutoModel, AutoModelWithLMHead,GPT2LMHeadModel
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import pickle, glob, json, csv

Import the GPT2 model from https://huggingface.co/LorenzoDeMattei/GePpeTto Paper: https://arxiv.org/abs/2004.14253 "GePpeTto Carves Italian into a Language Model"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('LorenzoDeMattei/GePpeTto')

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/534k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/280k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [ ]:

model = GPT2LMHeadModel.from_pretrained("LorenzoDeMattei/GePpeTto").to('cuda')

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/534k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/280k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/463M [00:00<?, ?B/s]

Import the text of the narrative "gianna_orig.txt". This file contain the full text of the narrative including punctuation marks

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
f=open('/content/drive/My Drive/Colab Notebooks/GPT/gianna_noVirgoletteParentesi_v02.txt', 'r',encoding='utf-8-sig')
orig_text = f.read() # the text is read as whole

In [ ]:
full_words = orig_text.split(' ')

#print(full_words)

# initialize a null list
unique_words = []
  
# traverse for all elements
for x in full_words:
    # check if exists in unique_list or not
    if x not in unique_words:
        unique_words.append(x.lower())

print(unique_words)

['in', 'principio,', 'avevo', 'solo', 'due', 'chiavi', 'quella', 'della', 'porta', 'di', 'casa', 'e', 'del', 'portone', 'palazzo.', 'ma', 'non', 'lavoro.', 'facevo', 'un', 'part', 'time', 'nero', 'negozio', 'giocattoli,', 'figuriamoci', 'se', 'potevo', 'pretendere', 'che', 'mi', 'dessero', 'le', 'chiavi.', 'e', 'poi,', 'scusate', 'che?', 'era', 'uno', 'quei', 'grandi', 'magazzini', 'sulla', 'variante,', 'sembrano', 'sempre', 'aperti,', 'o', 'almeno', 'li', 'vedete', 'mai', 'chiusi,', 'con', 'porte', 'automatiche', 'tutto.', 'che', 'bisogno', "c'era", 'una', 'chiave?', 'poi,', 'sapete,', 'è', 'lavoro,', 'realtà', 'nemmeno', 'full', 'nero,', 'lo', 'si', 'dice', 'per', 'consolarsi', 'vado', 'al', 'poi', 'tutto', 'crolla', 'quando', 'qualcuno', 'diciamo', 'mia', 'mamma', 'voce', 'candida', "po'", 'sognante', 'quando', 'inizierai', 'a', 'lavorare...', 'come', 'volessi', 'lavorare.\nquindi,', 'delle', 'prime', 'cose,', 'anzi', 'forse', 'proprio', 'la', 'prima', 'il', 'dottor', 'vigliani', 'h

**Please note tha Goldestein et al. (2021) proved that the correlation between the predictions of the GPT2 and the human predictions increases as the context window increases by reaching an asymptote of r = .79 at 100-word context **
We could use a window of that size

We will try to use a fixed context of time (e.g. 30 s) instead of a fixed context of tokens

In [ ]:
all_tokens_timestamps = pd.read_csv('/content/drive/My Drive/Colab Notebooks/GPT/all_tokens+timestamp.txt',delimiter='\t')
timestamps=all_tokens_timestamps['time']
duration = all_tokens_timestamps['duration']

In [ ]:
input_ids = tokenizer.encode(orig_text, return_tensors="pt").to('cuda')

time_window = 15 #seconds
#we normalize respect the first word time. Thus the first word will have a timestamps equals to zero 
timestamps_zerobased = timestamps-timestamps[0]

#we need to find the index of the word at the begining of the time window
#we should first consider all the words from 0 to 15 seconds, then all the word
#after 30 seconds


#get the word at 30 seconds
first_word_id = np.abs(timestamps_zerobased-time_window).argmin()

#define variable for next word probabilities
next_word_probs = []

#model the token contained in the 30s time window (first window)
all_next_token_logits = model(input_ids[:,:first_word_id+1])[0].requires_grad_(True)

#loop until the penultimate word contained in the window to get sequential next word probs
for c_idx,c_word in enumerate(input_ids[0][:first_word_id+1]):
  
  word2pred = all_tokens_timestamps.iloc[c_idx]['token']
  context_words = all_tokens_timestamps.iloc[:c_idx]['token'].tolist()

  #softmax on the probabilities assigned to all words of the vocabulary given the context
  next_token_probs = F.softmax(all_next_token_logits[:,c_idx,:],dim=-1)
  next_word_probs.append([next_token_probs[:,input_ids[0][c_idx+1]].item(),
                         word2pred,
                         timestamps[first_word_id],
                         timestamps[c_idx],
                         duration[c_idx], 
                         context_words])
  




#when the first loop is finished we have analyzed the next-word probabilities
#of all the words contained in the window so we need to move the time-window


#loop on the remaining words

for i in range(first_word_id+1,input_ids.size()[1]-1): #skip the last period character
  
  curr_diff = timestamps_zerobased[i]-time_window
  start_idx = np.abs(timestamps_zerobased-curr_diff).argmin()

  word2pred = all_tokens_timestamps.iloc[i]['token']
  context_words = all_tokens_timestamps.iloc[start_idx:i]['token'].tolist()


  #consider only the tokens that fall in time window of 30 seconds
  all_next_token_logits = model(input_ids[:,start_idx:i])[0].requires_grad_(True)
  #perform a softmax on the logits assigned to each word of the vocabulary
  next_token_probs = F.softmax(all_next_token_logits[:,-1,:],dim=-1)
  #get the probability of the next word, the next word and the context words 
  next_word_probs.append([next_token_probs[:,input_ids[0][i+1]].item(),
                         word2pred,
                        timestamps[start_idx],
                        timestamps[i],
                        duration[i],
                        context_words])

In [ ]:
word_prob = pd.DataFrame({'token_predicted' : [tmp[1] for tmp in next_word_probs],
                          'probability':np.array([tmp[0] for tmp in next_word_probs]),
                          'duration':np.array([tmp[4] for tmp in next_word_probs]),
                          'context_start_time':np.array([tmp[2] for tmp in next_word_probs]),
                          'context_stop_time':np.array([tmp[3] for tmp in next_word_probs])})

f = open('/content/drive/My Drive/Colab Notebooks/GPT/raw_probabilities_context_words_win'+str(time_window)+'s.txt','w')

context_words = []

for tmp in next_word_probs:
    context_words.append(tmp[-1])
    for w in tmp[-1]:
      f.write(w +'\t')
    
    f.write('\n')
f.close()

word_prob.to_csv('/content/drive/My Drive/Colab Notebooks/GPT/raw_probabilities_win'+str(time_window)+'s.csv')

In [ ]:
word_prob.head(20)

,token_predicted,probability,duration,context_start_time,context_stop_time
0,In,0.000334,0.15,27.01,11.80
1,principio,0.064819,0.66,27.01,11.95
2,",",0.000491,0.09,27.01,12.61
3,avevo,0.013082,0.33,27.01,12.70
4,solo,0.032139,0.27,27.01,13.03
5,due,0.000318,0.18,27.01,13.30
6,chiavi,0.000005,0.45,27.01,13.48
7,quella,0.053213,0.30,27.01,14.35
8,della,0.042729,0.21,27.01,14.65
9,porta,0.058577,0.33,27.01,14.86


Multiply the probabilities associated to the tokens in which a single word has been divided. E.g. "ga"++"tto"

In [ ]:
import csv
tok2remove = []
with open('/content/drive/My Drive/Colab Notebooks/GPT/token2remove.txt') as csvfile:
   csvreader = csv.reader(csvfile, delimiter=',')
   for row in csvreader:
    tok2remove.append(row)

tok2remove = [int(item) for tmp in tok2remove for item in tmp]

#remove rows with multi tokens words
new_probs = word_prob.drop(tok2remove)

In [ ]:
new_probs.head(20)
new_probs.to_csv('/content/drive/My Drive/Colab Notebooks/GPT/clean_probabilities_win' + str(time_window) + 's.csv')

In [ ]:
clean_context_words = [context_words[i] for i in range(len(context_words)) if i not in tok2remove]

f = open('/content/drive/My Drive/Colab Notebooks/GPT/clean_probabilities_context_words_win' + str(time_window) + 's.txt','w')

for tmp in clean_context_words:
  
    for w in tmp:
      f.write(w +'\t')
    
    f.write('\n')
f.close()

Now we need to extract the saliency for each token of the test. The output would be likely a dictionary where the key is the token and the value is the salience vector for that token and the input sequence of tokens

In [ ]:
time_window = 15 #seconds

#we normalize respect the first word time. Thus the first word will have a timestamps equals to zero 
timestamps_zerobased = timestamps-timestamps[0]

#we need to find the index of the word at the begining of the time window
#we should first consider all the words from 0 to 30 seconds, then all the word
#after 30 seconds

#get the word at 30 seconds
first_word_id = np.abs(timestamps_zerobased-time_window).argmin()



#back to cpu
model = GPT2LMHeadModel.from_pretrained("LorenzoDeMattei/GePpeTto")#.to('cuda')
#get the embedding matrix
embedding_matrix = model.transformer.wte.weight
#then generate a one hot tensor to select only the input token for the considered token
vocab_size = embedding_matrix.shape[0]

#get the all the input ids
input_ids = tokenizer.encode(orig_text, return_tensors="pt")#.to('cuda')

#loop here with the sliding window 




#one hot tensor
one_hot_tensor = F.one_hot(input_ids[:,:first_word_id], num_classes=vocab_size).to(dtype=torch.float32).requires_grad_(True)

#embedding of the inputs
inputs_embeds = torch.matmul(one_hot_tensor, embedding_matrix).requires_grad_(True)
inputs_embeds.retain_grad()


token_saliency = []
#loop until the penultimate word contained in the window to get sequential next word probs
for c_idx,c_word in enumerate(input_ids[0][:first_word_id+1]):
  
  next_input_ids = input_ids[:,c_idx]
  
  word2pred = all_tokens_timestamps.iloc[c_idx]['token']
  context_words = all_tokens_timestamps.iloc[:c_idx]['token'].tolist()

  grad = torch.autograd.grad(model(inputs_embeds=inputs_embeds)[0][:,c_idx-1,next_input_ids],inputs_embeds)[0]
  grad_x_input = grad*inputs_embeds
  importance_raw = torch.norm(grad_x_input,dim=2)
  importance_norm = importance_raw/torch.sum(importance_raw)
  token_saliency.append([word2pred,
                           importance_norm,
                         0,
                         timestamps[c_idx],
                           context_words,
                         np.arange(0,c_idx)])

#dump the first 30 seconds
pickle.dump(token_saliency,open('/content/drive/My Drive/Colab Notebooks/GPT/gradients_first0.' + str(time_window) + 's.pkl','wb'))


Downloading:   0%|          | 0.00/463M [00:00<?, ?B/s]

In [ ]:
a=list(range(200,input_ids.size()[1]-first_word_id-1,first_word_id))
a.append(input_ids.size()[1])
print(a)

[200, 253, 306, 359, 412, 465, 518, 571, 624, 677, 730, 783, 836, 889, 942, 995, 1048, 1101, 1154, 1207, 1260, 1313, 1366, 1419, 1472, 1525, 1578, 1631, 1684, 1737, 1790, 1843, 1896, 1949, 2002, 2055, 2108, 2161, 2214, 2267, 2320, 2373, 2426, 2512]


In [ ]:
#the real sliding window procedure after the first window
#loop on the remaining words

#to avoid a memory issue we define a set of index at which we dump the results and clean the output variable
parts = list(range(200,input_ids.size()[1]-first_word_id-1,first_word_id))
parts.append(input_ids.size()[1])

#define output variable
token_saliency = []

for i in range(first_word_id+1,input_ids.size()[1]-1):
  
  #get the initial time of the context
  curr_diff = timestamps_zerobased[i]-time_window
  start_idx = np.abs(timestamps_zerobased-curr_diff).argmin()

  word2pred = all_tokens_timestamps.iloc[i]['token'] #we want to predict the i-th word, thus the context is up to the i-th with the i-th excluded
  context_words = all_tokens_timestamps.iloc[start_idx:i]['token'].tolist()

  #id of the word to predict
  next_input_ids = input_ids[:,i]

  #one hot tensor for the tokens contained in the window
  one_hot_tensor = F.one_hot(input_ids[:,start_idx:i], num_classes=vocab_size).to(dtype=torch.float32).requires_grad_(True)


  #embedding of the inputs
  inputs_embeds = torch.matmul(one_hot_tensor, embedding_matrix).requires_grad_(True)
  inputs_embeds.retain_grad()

  grad = torch.autograd.grad(model(inputs_embeds=inputs_embeds)[0][:,-1,next_input_ids],
                             inputs_embeds)[0]
  grad_x_input = grad*inputs_embeds
  importance_raw = torch.norm(grad_x_input,dim=2)
  importance_norm = importance_raw/torch.sum(importance_raw)
  token_saliency.append([word2pred,
                           importance_norm,
                           timestamps[start_idx],
                          timestamps[i],
                           context_words,
                         np.arange(start_idx,i)])                                                     

  if i in parts:
    print('dump',i)
    print('length',len(token_saliency))
    pickle.dump(token_saliency,open('/content/drive/My Drive/Colab Notebooks/GPT/gradients_upto'+str(timestamps[i])+'words.pkl','wb'))
    token_saliency = []

#last token    
pickle.dump(token_saliency,open('/content/drive/My Drive/Colab Notebooks/GPT/gradients_upto'+str(timestamps[i])+'words.pkl','wb'))



#estimate the gradient*input that gives the importance of each input token for the selected output 
#https://jalammar.github.io/explaining-transformers/ https://github.com/jalammar/ecco/blob/main/src/ecco/attribution.py 
#A Diagnostic Study of Explainability Techniques for Text Classification;
#The elephant in the interpretability room: Why use attention as explanation when we have saliency methods?


dump 200
length 147
dump 253
length 53
dump 306
length 53
dump 359
length 53
dump 412
length 53
dump 465
length 53
dump 518
length 53
dump 571
length 53
dump 624
length 53
dump 677
length 53
dump 730
length 53
dump 783
length 53
dump 836
length 53
dump 889
length 53
dump 942
length 53
dump 995
length 53
dump 1048
length 53
dump 1101
length 53
dump 1154
length 53
dump 1207
length 53
dump 1260
length 53
dump 1313
length 53
dump 1366
length 53
dump 1419
length 53
dump 1472
length 53
dump 1525
length 53
dump 1578
length 53
dump 1631
length 53
dump 1684
length 53
dump 1737
length 53
dump 1790
length 53
dump 1843
length 53
dump 1896
length 53
dump 1949
length 53
dump 2002
length 53
dump 2055
length 53
dump 2108
length 53
dump 2161
length 53
dump 2214
length 53
dump 2267
length 53
dump 2320
length 53
dump 2373
length 53
dump 2426
length 53


In [ ]:
saliency_files = glob.glob('/content/drive/My Drive/Colab Notebooks/GPT/NoQuotMarks_timewindow' + str(time_window) + '/*gradients*.pkl')
saliency_file = saliency_files.sort()
print(saliency_files)

['/content/drive/My Drive/Colab Notebooks/GPT/NoQuotMarks_timewindow15/gradients_first0.15s.pkl', '/content/drive/My Drive/Colab Notebooks/GPT/NoQuotMarks_timewindow15/gradients_upto070.03words.pkl', '/content/drive/My Drive/Colab Notebooks/GPT/NoQuotMarks_timewindow15/gradients_upto084.94words.pkl', '/content/drive/My Drive/Colab Notebooks/GPT/NoQuotMarks_timewindow15/gradients_upto101.11words.pkl', '/content/drive/My Drive/Colab Notebooks/GPT/NoQuotMarks_timewindow15/gradients_upto114.49words.pkl', '/content/drive/My Drive/Colab Notebooks/GPT/NoQuotMarks_timewindow15/gradients_upto131.22words.pkl', '/content/drive/My Drive/Colab Notebooks/GPT/NoQuotMarks_timewindow15/gradients_upto144.93words.pkl', '/content/drive/My Drive/Colab Notebooks/GPT/NoQuotMarks_timewindow15/gradients_upto159.26words.pkl', '/content/drive/My Drive/Colab Notebooks/GPT/NoQuotMarks_timewindow15/gradients_upto173.16words.pkl', '/content/drive/My Drive/Colab Notebooks/GPT/NoQuotMarks_timewindow15/gradients_upto18

In [ ]:
#load and store saliency vector

pred_words = []
raw_saliency_matrix = []
start_times = []
stop_times = []
past_tokens = []
past_tokens_indices = []


for tmp_file in saliency_files:
    tmp_list = pickle.load(open(tmp_file,'rb'))

    for val in tmp_list:
      pred_words.append(val[0])
      raw_saliency_matrix.append(val[1].detach().numpy())
      start_times.append(val[2])
      stop_times.append(val[3])
      past_tokens.append(val[4])
      past_tokens_indices.append(val[5])

In [ ]:
#save the previous outputs
f = open('/content/drive/My Drive/Colab Notebooks/GPT/raw_salience_win' + str(time_window) + 's.txt',"w")

for i,vec in enumerate(raw_saliency_matrix):
  f.write(pred_words[i] + ',' + str(start_times[i]) + ',' + str(stop_times[i]) + ',')
  for el in vec:
    f.write(str(el))
  f.write('\n')

f.close()

f = open('/content/drive/My Drive/Colab Notebooks/GPT/raw_past_tokens_win' + str(time_window) + 's.txt',"w")

for i,tmp_list in enumerate(past_tokens):
  f.write(pred_words[i]+'\t')
  for val in tmp_list:
    f.write(val + '\t')
  f.write('\n')

f.close()


#save also as pickle file
raw_salience_list = [[pred_words[i],start_times[i],stop_times[i],vec,past_tokens_indices[i]]
                     for i,vec in enumerate(raw_saliency_matrix)]
raw_past_token_list =[[pred_words[i],tmp_list]
                      for i,tmp_list in enumerate(past_tokens)]

pickle.dump(raw_salience_list,open('/content/drive/My Drive/Colab Notebooks/GPT/time_window/30s/raw_salience_win' + str(time_window) + 's.pkl',"wb")) 
pickle.dump(raw_past_token_list,open('/content/drive/My Drive/Colab Notebooks/GPT/time_window/30s/raw_past_token_win' + str(time_window) + 's.pkl',"wb"))                      

Now we need to clean the clean the saliency matrix to select only the single token word

In [ ]:
#load the indices of the elements to remove
import csv
tok2remove = []
with open('/content/drive/My Drive/Colab Notebooks/GPT/token2remove.txt') as csvfile:
   csvreader = csv.reader(csvfile, delimiter=',')
   for row in csvreader:
    tok2remove.append(row)
tok2remove = [int(item) for tmp in tok2remove for item in tmp]


clean_saliency_matrix = [raw_saliency_matrix[i] for i in range(len(raw_saliency_matrix)) if i not in tok2remove]
clean_past_tokens = [past_tokens[i] for i in range(len(past_tokens)) if i not in tok2remove]
clean_starts = [start_times[i] for i in range(len(start_times)) if i not in tok2remove]
clean_stops = [stop_times[i] for i in range(len(stop_times)) if i not in tok2remove]
clean_pred_words = [pred_words[i] for i in range(len(pred_words)) if i not in tok2remove]
clean_past_token_indices = [past_tokens_indices[i] for i in range(len(past_tokens_indices)) if i not in tok2remove]

#save the clean output
f = open('/content/drive/My Drive/Colab Notebooks/GPT/clean_salience_win' + str(time_window) + 's.txt',"w")

for i,vec in enumerate(clean_saliency_matrix):
  f.write(clean_pred_words[i] + '\t' + str(clean_starts[i]) + '\t' + str(clean_stops[i]) + '\t' + str(vec))
  f.write('\n')

f.close()

f = open('/content/drive/My Drive/Colab Notebooks/GPT/clean_past_tokens_win' + str(time_window) + 's.txt',"w")

for i,tmp_list in enumerate(clean_past_tokens):
  f.write(clean_pred_words[i]+'\t')
  for val in tmp_list:
    f.write(val + '\t')
  f.write('\n')

f.close()

#save also as pickle file
clean_salience_list = [[clean_pred_words[i],clean_starts[i],clean_stops[i],vec,clean_past_token_indices[i]]
                     for i,vec in enumerate(clean_saliency_matrix)]
clean_past_token_list =[[clean_pred_words[i],tmp_list]
                      for i,tmp_list in enumerate(clean_past_tokens)]

pickle.dump(clean_salience_list,open('/content/drive/My Drive/Colab Notebooks/GPT/time_window/' + str(time_window) + 's/clean_salience_win' + str(time_window) + 's.pkl',"wb")) 
pickle.dump(clean_past_token_list,open('/content/drive/My Drive/Colab Notebooks/GPT/time_window/' + str(time_window) + 's/clean_past_token_win' + str(time_window) + 's.pkl',"wb")) 